In [78]:
import pandas as pd
import json
import requests
import datetime as dt
from datetime import date
from dateutil.parser import *

In [3]:
url = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Use_of_Force/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [4]:
response = requests.get(url)

In [5]:
response_json = response.json()

In [8]:
response_json.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'features'])

In [28]:
response_json['fields']

[{'name': 'PoliceUseOfForceID',
  'type': 'esriFieldTypeInteger',
  'alias': 'PoliceUseOfForceID',
  'sqlType': 'sqlTypeOther',
  'domain': None,
  'defaultValue': None},
 {'name': 'CaseNumber',
  'type': 'esriFieldTypeString',
  'alias': 'CaseNumber',
  'sqlType': 'sqlTypeOther',
  'length': 100,
  'domain': None,
  'defaultValue': None},
 {'name': 'ResponseDate',
  'type': 'esriFieldTypeDate',
  'alias': 'ResponseDate',
  'sqlType': 'sqlTypeOther',
  'length': 8,
  'domain': None,
  'defaultValue': None},
 {'name': 'Problem',
  'type': 'esriFieldTypeString',
  'alias': 'Problem',
  'sqlType': 'sqlTypeOther',
  'length': 400,
  'domain': None,
  'defaultValue': None},
 {'name': 'Is911Call',
  'type': 'esriFieldTypeString',
  'alias': 'Is911Call',
  'sqlType': 'sqlTypeOther',
  'length': 3,
  'domain': None,
  'defaultValue': None},
 {'name': 'PrimaryOffense',
  'type': 'esriFieldTypeString',
  'alias': 'PrimaryOffense',
  'sqlType': 'sqlTypeOther',
  'length': 8,
  'domain': None,
  '

In [18]:
len(response_json["features"])

33721

In [21]:
response_json["features"][0]['attributes'].keys()

dict_keys(['PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem', 'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber', 'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction', 'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct', 'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear', 'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude', 'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'])

In [111]:
new_dict = {}
for field in response_json["features"][0]['attributes'].keys():
    new_dict[field] = []
for index in range(len(response_json["features"])):
    a = response_json["features"][index]['attributes']
    for key in response_json["features"][index]['attributes']:
#         print(key, a[key])
        if key == 'ResponseDate':
            d = datetime.datetime.fromtimestamp(a[key]/1000)
            e = d.strftime('%Y-%m-%d %H:%M:%S')
            new_dict[key].append(e)
        else:
            new_dict[key].append(a[key])
    if index > 10000:
        break
df = pd.DataFrame(new_dict)

In [112]:
df.head()

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,SubjectRole,SubjectRoleNumber,...,TotalCityCallsForYear,TotalPrecinctCallsForYear,TotalNeighborhoodCallsForYear,CenterGBSID,CenterLatitude,CenterLongitude,CenterX,CenterY,DateAdded,OBJECTID
0,24375067,08-003643,2008-01-04 04:40:37,Suspicious Vehicle,No,AOA,No,1,OT,1,...,322402,55689,841,18837,44.925909,-93.296905,-1.038576e+07,5.609865e+06,1626509939000,1
1,24375068,08-010291,2008-01-10 16:21:53,Motor Vehicle Chase,No,FLEE,Yes,6,A,1,...,322402,84018,10923,11907,44.943185,-93.265782,-1.038230e+07,5.612582e+06,1626509939000,2
2,24375069,08-018909,2008-01-19 16:04:01,Suspected Hazard,No,ASLT5,Yes,1,A,1,...,322402,46998,23458,17476,44.978407,-93.278819,-1.038375e+07,5.618123e+06,1626509939000,3
3,24375070,08-032626,2008-02-02 16:40:31,Motor Vehicle Chase,No,FLEE,Yes,2,A,1,...,322402,80434,10077,16331,44.984396,-93.304503,-1.038661e+07,5.619065e+06,1626509939000,4
4,24375071,08-038886,2008-02-08 20:11:03,Fight,No,DISCON,None,2,A,2,...,322402,46998,23458,25832,44.979626,-93.272499,-1.038305e+07,5.618315e+06,1626509939000,5


In [113]:
df.Problem.value_counts()

Suspicious Person              1976
Fight                          1117
Disturbance                     836
Domestic Abuse-In Progress      589
Suspicious Vehicle              499
                               ... 
Kidnapping/Abduction              1
PI w/trapped                      1
Aircraft Crash in City            1
Down Outide-One w/Fire            1
Overdose-Accidental               1
Name: Problem, Length: 95, dtype: int64

In [117]:
df.PrimaryOffense.value_counts()

OBSTRU    2133
DISCON    1250
ASLT4      838
NARC       535
FORCE      531
          ... 
HOUSNG       1
XSOUND       1
IPROST       1
OBNARC       1
INCEND       1
Name: PrimaryOffense, Length: 164, dtype: int64

In [116]:
df.Neighborhood.value_counts()

Downtown West           2770
Jordan                   542
Hawthorne                473
Near - North             420
Willard - Hay            403
                        ... 
Kenwood                    6
Bryn - Mawr                5
Armatage                   2
Cedar - Isles - Dean       1
Kenny                      1
Name: Neighborhood, Length: 86, dtype: int64

In [115]:
df.Race.value_counts(1)

Black                 0.627375
White                 0.241952
Native American       0.051690
Other / Mixed Race    0.044991
Asian                 0.017796
Unknown               0.008798
not recorded          0.007099
Pacific Islander      0.000300
Name: Race, dtype: float64

In [22]:
response_json["features"][0]['geometry']

{'x': -93.2969048626618, 'y': 44.92590912407578}

In [32]:
b = response_json["features"][0]['attributes']['ResponseDate']

In [ ]:
# Map Data Table - CaseNumber, Lat, Long, Neighborhood, Precinct
# Incident Data Table - everything else

In [29]:
a = "1/4/2008, 4:40 AM"

In [31]:
parse(a)

datetime.datetime(2008, 1, 4, 4, 40)

In [51]:
dt.datetime.fromtimestamp(b/1000)

datetime.datetime(2008, 1, 10, 16, 21, 53)

In [53]:
import datetime
d = datetime.datetime.fromtimestamp(b/1000)
print(d.strftime('%Y-%m-%d %H:%M:%S'))

2008-01-10 16:21:53


In [55]:
d.hour

16

In [57]:
d.weekday()

3

In [59]:
b=1199330424000

In [60]:
d = datetime.datetime.fromtimestamp(b/1000)
print(d.strftime('%Y-%m-%d %H:%M:%S'))

2008-01-02 21:20:24


In [58]:
response_json["features"][500]

{'attributes': {'PoliceUseOfForceID': 24375567,
  'CaseNumber': '08-002346',
  'ResponseDate': 1199330424000,
  'Problem': 'Theft ',
  'Is911Call': 'Yes',
  'PrimaryOffense': 'BURGTL',
  'SubjectInjury': 'No',
  'ForceReportNumber': 1,
  'SubjectRole': 'A',
  'SubjectRoleNumber': 3,
  'ForceType': 'Bodily Force',
  'ForceTypeAction': 'Body Weight to Pin',
  'Race': 'Asian',
  'Sex': 'Male',
  'EventAge': 14,
  'TypeOfResistance': 'Fled on Foot',
  'Precinct': '04',
  'Neighborhood': 'Sumner - Glenwood',
  'TotalCityCallsForYear': 322402,
  'TotalPrecinctCallsForYear': 80434,
  'TotalNeighborhoodCallsForYear': 1123,
  'CenterGBSID': 16516,
  'CenterLatitude': 44.98883,
  'CenterLongitude': -93.29104,
  'CenterX': -10385111.0281,
  'CenterY': 5619763.1442,
  'DateAdded': 1626509939000,
  'OBJECTID': 501},
 'geometry': {'x': -93.29103963820427, 'y': 44.98882983812188}}